<img width="8%" alt="Pipedrive.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Pipedrive.png" style="border-radius: 15%">

# Pipedrive - Get all notes
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Pipedrive+-+Get+all+notes:+Error+short+description">Bug report</a>

**Tags:** #pipedrive #notes #get #snippet #api #v1 #python #csv #export

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-01-04 (Created: 2023-01-04)

**Description:** This notebook retrieves all notes with access to your Pipedrive account.

**References:**
- [Pipedrive API v1 Documentation](https://developers.pipedrive.com/docs/api/v1/Notes#getNotes)
- [Pipedrive API Authentication](https://developers.pipedrive.com/docs/api/authentication)
- [Get your Pipedrive API token](https://developers.pipedrive.com/docs/api/authentication).

## Input

### Import libraries

In [ ]:
import requests
import naas
import pandas as pd
from datetime import date

### Setup variables
- `api_token`: API token used to authenticate the request.
- `file_path`: CSV file path to save data
- `params`: query parameters

In [ ]:
api_token = naas.secret.get("PIPEDRIVE_API_KEY") or "YOUR_API_TOKEN"
file_path = f"{date.today().isoformat()}_export_pipedrive_notes.csv"
params = {
    "user_id": None,
    "lead_id": None,
    "deal_id": None,
    "person_id": None,
    "org_id": None,
}

## Model

### Get all notes

In [ ]:
# Flatten the nested dict
def flatten_dict(d, parent_key='', sep='_'):
    """
    Flattens a nested dictionary into a single level dictionary.

    Args:
        d (dict): A nested dictionary.
        parent_key (str): Optional string to prefix the keys with.
        sep (str): Optional separator to use between parent_key and child_key.

    Returns:
        dict: A flattened dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def get_all_notes(
    api_token,
    params,
    start=0,
    limit=100,
):
    # Init
    data = []
    
    # Requests
    while True:
        api_url = f'https://api.pipedrive.com/v1/notes?api_token={api_token}&start={start}&limit={limit}'
        res = requests.get(api_url, params)
        res.raise_for_status()
        if res.status_code == 200:
            res_json = res.json()
        else:
            print(f'Failed to get all notes. Status code: {res.status_code}')
            break
            
        # Clean data
        if res_json.get("success") and res_json.get('data') is not None:
            for r in res_json.get('data'):
                data.append(flatten_dict(r))
            start += 100
        else:
            break
    return pd.DataFrame(data)
        
df = get_all_notes(api_token, params)
print("Notes:", len(df))
df.head(5)

## Output

### Save data to CSV

In [ ]:
df.to_csv(file_path, index=False)